# Init

In [69]:
import pandas as pd
import numpy as np
import os
import math
import matplotlib.pyplot as plt
import glob
from scipy.spatial.distance import euclidean, pdist, squareform
import sys
import sys
import os
sys.path.append(os.environ['DEV_AUTOTS'])
sys.path.append(os.environ['CAPSTONE_PYTHON_SOURCE'])
folder = os.environ['CAPSTONE_DATA']


In [70]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [71]:
from ETL.ETL import loadDataset, getTopProducts
from similarity.similarity import mergeTopSimilar, loadSimilarity
dataRaw= loadDataset(version=4)
dfSimilarity = loadSimilarity(version=4)


# Prep Data

In [72]:
#Parameters
ChainMaster = 'SPECS'
ProdCat='SUP PREM WHISKEY'
ProductsList = getTopProducts(dataRaw,ChainMaster=ChainMaster,ProdCat=ProdCat,topN=5,timeCol='WeekDate')
ProductsList

['JACK DANIELS BLK WHSKY  1L',
 'JACK DANIELS BLK WHSKY  1.75L',
 'JACK DANIELS BLK WHSKY  750M',
 'BUFFALO TRACE BOURBON  1L',
 'WL WELLER BBN RSV  1.75L']

In [73]:

Product = ProductsList[0]
colTime = 'WeekDate'
colPreds = ['9L Cases'] #Demand
colPrice = ['Dollar Sales per 9L Case'] #Price

#TO DO:
#Merge top similar only looking at a list of products  DONE
#Encode before the merge? (same prods same code) DONE
#one chain, top three products, top 1 similar DONE
#Impute the price DONE
(dataModel,colExog,colEnc,colDec) = mergeTopSimilar(dataRaw,dfSimilarity
                                                    ,ChainMaster=ChainMaster
                                                    ,Product=Product
                                                    ,ProductsList=ProductsList
                                                    ,topn=2 
                                                    ,selCols = colPrice + colPreds,periodCol = colTime
                                                    ,pivotCol='Dollar Sales per 9L Case' 
                                                    ,encodeCols=True)
                                                                        
#colExog = colExog + colEndog
print(colExog)
print(colDec)
allCols=[colTime]+colPreds+ colExog
data=dataModel[allCols]
data.tail(5)

['0', '1', '2']
{'0': 'JACK DANIELS BLK WHSKY  1L', '1': 'JACK DANIELS BLK WHSKY  1.75L', '2': 'JACK DANIELS BLK WHSKY  750M', '3': 'BUFFALO TRACE BOURBON  1L', '4': 'WL WELLER BBN RSV  1.75L'}


,WeekDate,9L Cases,0,1,2
360,2019-11-28,34.58,229.533835,185.589744,185.589744
361,2019-12-05,37.24,229.533835,185.589744,185.589744
362,2019-12-12,159.60,229.533835,185.589744,222.360000
363,2019-12-19,35.91,229.533835,185.589744,222.360000
364,2019-12-26,243.39,229.533835,185.589744,222.360000


In [82]:
train = data.iloc[:-4]
test = data.iloc[-4:]
print(train.shape,test.shape)

(361, 5) (4, 5)


# Model

In [83]:
import auto_ts as AT
automl_model = AT.AutoTimeSeries(
    score_type='rmse', forecast_period=3, time_interval='Week',
    non_seasonal_pdq=None, seasonality=True, seasonal_period=12,
    model_type=['SARIMAX','ML'],
    verbose=0)
automl_model.fit(train, 'WeekDate', '9L Cases', ',')



Start of Fit.....
    Target variable = 9L Cases
Start of loading of data.....
Input is data frame. Performing Time Series Analysis
    Dataframe loaded successfully. Shape of data set = (361, 4)
Time Interval is given as Week


Building SARIMAX Model



Running Seasonal SARIMAX Model...
Building a Seasonal Model...

    Finding best Non-Seasonal pdq Parameters:

Differencing = 0 with Seasonality = False
    Iteration 0 completed...
Inside find_best_pdq_or_PDQ --> results_bic
             MA0          MA1          MA2          MA3
AR0          NaN  3573.230414  3558.635427  3542.791908
AR1  3590.528299  3565.160696  3568.187820  3544.784693
AR2  3578.986458  3564.321022  3557.793868  3550.745331
AR3  3559.726451  3561.618528  3559.671861  3550.798272
    Best AR order p = 0, MA order q = 3, Interim metric = 3542.792

Differencing = 1 with Seasonality = True
Inside find_best_pdq_or_PDQ --> results_bic
     MA0  MA1  MA2  MA3
AR0  NaN  NaN  NaN  NaN
AR1  NaN  NaN  NaN  NaN
AR2  NaN  NaN 

In [84]:
automl_model.get_leaderboard()

,name,rmse
0,SARIMAX,23.102260
1,ML,25.665994


# Prediction

In [85]:
test_predictions = automl_model.predict(X_exogen = test)  # TODO: pass the test dataframe (exogen) - can add model = to to get a model speicifc
test_predictions

2019-12-05    47.816210
2019-12-12    38.188549
2019-12-19    56.237826
2019-12-26    59.223579
Freq: W-THU, Name: mean, dtype: float64

In [87]:
test_predictions = automl_model.predict(X_exogen = test,model='ML')  # TODO: pass the test dataframe (exogen) - can add model = to to get a model speicifc
print(test_predictions)
test_predictions = automl_model.predict(X_exogen = test.iloc[0:3][colExog ],model='SARIMAX')  # TODO: pass the test dataframe (exogen) - can add model = to to get a model speicifc
print(test_predictions)
test

0    49.066614
1    49.179621
2    49.170273
3    49.258039
Name: mean, dtype: float64
2019-12-05    47.816210
2019-12-12    38.188549
2019-12-19    56.237826
Freq: W-THU, Name: mean, dtype: float64


,WeekDate,9L Cases,0,1,2
361,2019-12-05,37.24,229.533835,185.589744,185.589744
362,2019-12-12,159.60,229.533835,185.589744,222.360000
363,2019-12-19,35.91,229.533835,185.589744,222.360000
364,2019-12-26,243.39,229.533835,185.589744,222.360000
